In [7]:
import sys
sys.path.append("../../src/earthtext")

from osm import osm
import os
from progressbar import progressbar as pbar
import shapely as sh 
from pyproj import CRS
import numpy as np
epsg4326 = CRS.from_epsg(4326)
import geopandas as gpd
import pandas as pd
from importlib import reload
from rlxutils import Command, mParallel
from joblib import delayed, Parallel
from time import time
import folium
reload(osm)

<module 'osm.osm' from '/home/ubuntu/earth-text/notebooks/osm/../../src/earthtext/osm/osm.py'>

In [9]:
dg = gpd.read_file("/opt/data/california-worldcover-chips.fgb")
dg['chip_id'] = [osm.get_region_hash(i) for i in dg.geometry.values]

In [10]:
# get only chips with data
dgd = dg[[os.path.isfile(f"/opt/data/california-worldcover-chips/osm/{i}.parquet") for i in dg.chip_id]]
dgd.shape

(93363, 4)

In [96]:
def process_chipid(chip_id, geometry):
    fname = f"/opt/data/california-worldcover-chips/osm/{chip_id}.parquet"
    
    z = gpd.read_parquet(fname)
    if len(z)==0:
        return "no_geoms"

    # in case it was processed before
    z['tags'] = [eval(t) if isinstance(t, str) else t for t in z.tags]
    
    z['tags'] = [{k:v for k,v in t.items() if v is not None} for t in z.tags]
    z['tags'] = [osm.clean_tags(t) for t in z.tags]
    z = z[[len(t)>0 for t in z.tags]]
    
    if len(z)==0:
        return "no_tags"

    g = geometry
    z['stags'] = [osm.tags2str(t) for t in z.tags]
    z['chip_id'] = chip_id

    # dictionaries in parquet seem to be scrambled
    # when loading do z['tags'] = [eval(t) for t in z.tags]
    z['tags'] = [str(t) for t in z.tags]
    z['chip_id'] = chip_id
    
    zi = z[[i.intersects(g) for i in z.geometry]]
    zi['geometry'] = [i.intersection(g) for i in zi.geometry]
    zic = zi.to_crs(osm.epsg_california)
    zi['area'] = [g.area for g in zic.geometry]
    zi['length'] = [g.length for g in zic.geometry]
    zi.to_parquet(fname)
    return "ok"

class mParallel(Parallel):
    """
    substitutes joblib.Parallel with richer verbose progress information
    """
    def _print(self, msg):
        if self.verbose > 10:
            fmsg = '[%s]: %s' % (self, msg)
            sys.stdout.write('\r ' + fmsg)
            sys.stdout.flush()


In [97]:
try:
    chip_ids = dgd.chip_id.values.copy()
    geometries = dgd.geometry.values.copy()
    del (dgd)
except:
    pass

In [98]:
r = mParallel(n_jobs=20, verbose=30)(delayed(process_chipid)(c,g) for c, g in zip(chip_ids, geometries))

 [mParallel(n_jobs=20)]: Done 93363 out of 93363 | elapsed:  4.2min finishedSetting batch_size=8.

In [101]:
def process_chipid(chip_id, geometry):
    fname = f"/opt/data/california-worldcover-chips/osm/{chip_id}.parquet"
    z = gpd.read_parquet(fname)
    if len(z)==0:
        return "no_geoms"
    z['tags'] = [{k:v for k,v in t.items() if v is not None} for t in z.tags]
    z['tags'] = [osm.clean_tags(t) for t in z.tags]
    z = z[[len(t)>0 for t in z.tags]]
    
    if len(z)==0:
        if os.path.isfile(fname):
            os.remove(fname)
        return "no_tags"

    g = geometry
    z['stags'] = [osm.tags2str(t) for t in z.tags]

    # dictionaries in parquet seem to be scrambled
    # when loading do z['tags'] = [eval(t) for t in z.tags]
    z['tags'] = [str(t) for t in z.tags]
    z['chip_id'] = chip_id

    zi = z[[i.intersects(g) for i in z.geometry]]

    
    zi['geometry'] = [i.intersection(g) for i in zi.geometry]
    zic = zi.to_crs(osm.epsg_california)
    zi['area'] = [g.area for g in zic.geometry]
    zi['length'] = [g.length for g in zic.geometry]
    zi.to_parquet(fname)
    return "ok"

In [ ]:
20bf05e790bfd

In [84]:
reload(osm)

<module 'osm.osm' from '/home/ubuntu/earth-text/notebooks/osm/../../src/earthtext/osm/osm.py'>

In [108]:
chip_id = '3532a7debbf63'
geom = [geometries[i] for i in range(len(geometries)) if chip_ids[i]==chip_id][0]


In [109]:
process_chipid(chip_id, geom)

FileNotFoundError: /opt/data/california-worldcover-chips/osm/3532a7debbf63.parquet

In [87]:
fname = '/opt/data/california-worldcover-chips/osm/33c66a6cd3ee7.parquet'

In [67]:
!rm $fname

In [89]:
z = gpd.read_parquet(fname)
z['tags'] = [eval(t) for t in z.tags]
z.head()

,tags,geometry,kind,length,area,stags,chip_id
7226761673,{'barrier': 'gate'},POINT (-120.50629 37.36087),node,0.0,0.0,barrier: gate,33c66a6cd3ee7
7401789632,{'barrier': 'gate'},POINT (-120.50379 37.35729),node,0.0,0.0,barrier: gate,33c66a6cd3ee7
8047894562,{'barrier': 'gate'},POINT (-120.50459 37.35693),node,0.0,0.0,barrier: gate,33c66a6cd3ee7
8272942981,{'barrier': 'gate'},POINT (-120.49887 37.36562),node,0.0,0.0,barrier: gate,33c66a6cd3ee7
8272942989,{'barrier': 'gate'},POINT (-120.49886 37.36476),node,0.0,0.0,barrier: gate,33c66a6cd3ee7


In [92]:
z.tags.values[0]['barrier']

'gate'